<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/Metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install wandb -qU
!wandb login 617d80dc70f383d8ff9ef1c2b94a78ccc8d8db23

     |████████████████████████████████| 1.9 MB 5.3 MB/s 
     |████████████████████████████████| 168 kB 47.4 MB/s 
     |████████████████████████████████| 182 kB 56.9 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 168 kB 71.8 MB/s 
     |████████████████████████████████| 166 kB 72.6 MB/s 
     |████████████████████████████████| 166 kB 68.5 MB/s 
     |████████████████████████████████| 162 kB 69.0 MB/s 
     |████████████████████████████████| 162 kB 47.5 MB/s 
     |████████████████████████████████| 158 kB 64.7 MB/s 
     |████████████████████████████████| 157 kB 68.3 MB/s 
     |████████████████████████████████| 157 kB 45.1 MB/s 
     |████████████████████████████████| 157 kB 63.8 MB/s 
     |████████████████████████████████| 157 kB 69.5 MB/s 
     |████████████████████████████████| 157 kB 69.2 MB/s 
     |████████████████████████████████| 157 kB 69.6 MB/s 
     |████████████████████████████████| 157 kB 56.9 MB/s 
     |███████████

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
import numpy as np
from sklearn.metrics import classification_report, f1_score

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  shuffle = False,
  image_size=(img_height, img_width))

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 100 files belonging to 5 classes.


In [6]:
y = np.concatenate([y for x, y in test_ds], axis=0)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4], dtype=int32)

In [7]:
labels = test_ds.class_names

In [8]:
class argmax_layer(tf.keras.layers.Layer):
  def __init__(self):
    super(argmax_layer, self).__init__()

  def call(self, inputs):
    return tf.math.argmax(inputs, axis=1)

In [18]:
#vgg16
api = wandb.Api()
artifact = api.artifact('vedmanivaidya/Transfer_Learning_New/model-Xception_with_avgpooling:v63', type='model')
artifact.download()
model = tf.keras.models.load_model("/content/artifacts/model-Xception_with_avgpooling:v63")

wandb: Downloading large artifact model-Xception_with_avgpooling:v63, 83.10MB. 4 files... 
wandb:   4 of 4 files downloaded.  
Done. 0:0:1.4


In [19]:
model.evaluate(test_ds)

4/4 [==============================] - 20s 4s/step - loss: 0.1091 - accuracy: 0.9800


[0.1090865358710289, 0.9800000190734863]

In [20]:
seq = tf.keras.Sequential()
seq.add(model)
seq.add(argmax_layer())
preds = seq.predict(test_ds)

4/4 [==============================] - 20s 4s/step


In [21]:
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(y, preds, average='macro'))

(0.9894736842105264, 0.9772727272727273, 0.9828220193437585, None)


In [30]:
'''from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
f1_score(y, preds, average='macro')'''

"from sklearn.metrics import f1_score\nfrom sklearn.metrics import precision_recall_fscore_support\nf1_score(y, preds, average='macro')"